In [ ]:
##### DEFINE FUNCTIONS #####

In [1]:
## Get anomalies/composites
# Define anomaly function
# def get_anomaly(x,detrend):
#     clim = x.groupby('time.month').mean(dim='time')
#     if detrend == 1:
#         poly = x.polyfit(dim='time', deg=1)
#         fit = xr.polyval(x['time'], poly.polyfit_coefficients)
#         return x.groupby('time.month') - clim - fit
#     else:
#         return x.groupby('time.month') - clim

def get_anomaly(x,detrend):
    clim = x.groupby('time.month').mean(dim='time')
    anom = x.groupby('time.month') - clim
    if detrend == 1:
        poly = anom.polyfit(dim='time', deg=1)
        fit = xr.polyval(anom['time'], poly.polyfit_coefficients)
        return anom - fit
    else:
        return anom

# Define composite functions
def composite(x):
    path = '/data/keeling/a/tjhanke2/enso-energy-budget/Data/processed_data/ccf_anom_lo_res/nino3_4/'
    nino34 = xr.open_dataset(path+'nino34_index.nc').__xarray_dataarray_variable__
    
    return x.groupby(nino34.rename('nino34')).mean(dim='time')

In [ ]:
## Get global-means
# def global_mean(x):
#     return x.weighted(weight).mean(dim=('latitude','longitude'))
def global_mean(x):
    weights = np.cos(np.deg2rad(x.latitude))
    return x.weighted(weights).mean(dim=('latitude','longitude'))

In [ ]:
# ## Get Clear Sky Radiation Anomalies across pressure levels
# def get_mean_across_lev(kernel,del_param):
#     arr = []
#     kernel = kernel.transpose('plev',...)
#     del_param = del_param.transpose('plev',...)
    
#     for i in range(0,len(kernel.plev)):           
#         # Do calculation at each p level
#         if i != len(kernel.plev)-1:
#             calc = kernel[i] * del_param[i] * (kernel.plev[i] - kernel.plev[i+1])/100
#         else:
#             calc = kernel[i] * del_param[i] * (kernel.plev[i] - 0)/100
        
#         # Append to arrays
#         arr.append(calc)
    
#     # Concat each array into DataArrays, and then sum
#     calc_da = xr.concat(arr, dim='plev')
# #     calc_xr = calc_da.sum(dim='plev')
# #     calc_mon_avg = calc_xr.mean(dim='month')
#     calc_mon_avg = calc_da.mean(dim='month')
    
#     return calc_mon_avg

In [ ]:
## Get Clear Sky Radiation Anomalies across pressure levels
def get_mean_across_lev(del_param):
    arr = []
    del_param = del_param.transpose('plev',...)
    
    for i in range(0,len(del_param.plev)):           
        # Do calculation at each p level
        if i != len(del_param.plev)-1:
            calc = del_param[i] * (del_param.plev[i] - del_param.plev[i+1])/100
        else:
            calc = del_param[i] * (del_param.plev[i] - 0)/100
        
        # Append to arrays
        arr.append(calc)
    
    # Concat each array into DataArrays, and then sum
    calc_da = xr.concat(arr, dim='plev')
#     calc_xr = calc_da.sum(dim='plev')
#     calc_mon_avg = calc_xr.mean(dim='month')
#    calc_mon_avg = calc_da.mean(dim='month')
    
    return calc_da

In [ ]:
## Get CRE across p,tau bins
def get_mean_across_bins(x_cld,x_clr,cld_area):
    arr = []
    n = int(x_cld.press[0])
    m = int(x_cld.press[-1])
    
    for i in range(n,m):
        for j in range(0,6):            
            # Do calculation in each p,tau bin
            calc = cld_area.sel(press=i,opt=j) * (x_cld.sel(press=i,opt=j) - x_clr)
            
            # Append to arrays
            arr.append(calc)
    
    # Concat each array into DataArrays, and then sum
    calc_da = xr.concat(arr, dim='pressopt')
    calc_xr = calc_da.sum(dim='pressopt')
    
    return calc_xr

In [ ]:
# Get low cloud and high cloud CRE across p,tau bins
def get_mean_across_bins_loCld(x_cld,x_clr,cld_area):
    # Get low and high cloud fractions
    loCld_frac      = cld_area.sel(press=slice(0,1)).sum(dim=('press','opt'))
    highCld_frac    = cld_area.sel(press=slice(2,6)).sum(dim=('press','opt'))
    cld_frac_anom   = get_anomaly(cld_area,1)
    loCld_frac_anom = cld_frac_anom.sel(press=slice(0,1)).sum(dim=('press','opt'))
    highCld_frac_anom = cld_frac_anom.sel(press=slice(2,6)).sum(dim=('press','opt'))
    
    # Get climatology of cloud fraction
    cld_frac_clim   = cld_area.groupby('time.month').mean(dim='time')
    loCld_frac_clim = cld_frac_clim.sel(press=slice(0,1)).sum(dim=('press','opt'))
    highCld_frac_clim = cld_frac_clim.sel(press=slice(2,6)).sum(dim=('press','opt'))
    
    # Get non-obscured low cloud fraction
    non_obs_loCld         = loCld_frac / (1-highCld_frac)
    non_obs_loCld_anom    = get_anomaly(non_obs_loCld,1)
    non_obs_loCld_clim    = non_obs_loCld.groupby('time.month').mean(dim='time')
    non_obs_loCld_perturb = non_obs_loCld_anom.groupby('time.month') * (1-highCld_frac_clim)
#     non_obs_loCld_perturb = (loCld_frac_anom + highCld_frac_anom - highCld_frac_anom.groupby('time.month')*(1-non_obs_loCld_clim)).groupby('time.month') / (1-highCld_frac_clim)
    
    ### Get the radiative perturbation due to changes in non-obscured cloud fraction
    dR_dcld_frac = (((x_cld.sel(press=slice(0,1)) * cld_frac_clim.sel(press=slice(0,1)) / loCld_frac_clim).sum(dim=('opt','press')) - x_clr) * 
                    non_obs_loCld_perturb.groupby('time.month'))
    
    # Get L ratio
    loCld_ratio = loCld_frac_anom / loCld_frac_clim
    
    # Get cloud fraction anomaly that includes changes in p-tau distribution
    adjusted_cld_frac_anom = cld_frac_anom.groupby('time.month') - cld_frac_clim.sel(press=slice(0,1)) * loCld_ratio
    
    ### Get the radiative perturbation due to changes in the p-tau distribution
    dR_dcld_dist = ((x_cld - x_clr) * adjusted_cld_frac_anom.groupby('time.month')).sum(dim=('press','opt'))
        
    return xr.Dataset(data_vars = {'dR_dcld_frac':dR_dcld_frac, 'dR_dcld_dist':dR_dcld_dist})

In [ ]:
# Get low cloud and high cloud CRE across p,tau bins
def get_mean_across_bins_highCld(x_cld,x_clr,cld_area):
    # Get low and high cloud fractions
    loCld_frac   = cld_area.sel(press=slice(0,1)).sum(dim=('press','opt'))
    highCld_frac = cld_area.sel(press=slice(2,6)).sum(dim=('press','opt'))
    
    # Get non-obscured low cloud fraction
    highCld_anom    = get_anomaly(highCld_frac,1)
    
    # Get climatology of cloud fraction
    cld_frac_clim   = cld_area.groupby('time.month').mean(dim='time')
    highCld_frac_clim = cld_frac_clim.sel(press=slice(2,6)).sum(dim=('press','opt'))
    
    ### Get the radiative perturbation due to changes in non-obscured cloud fraction
    dR_dcld_frac = (((x_cld.sel(press=slice(2,6)) * cld_frac_clim.sel(press=slice(2,6)) / highCld_frac_clim).sum(dim=('opt','press')) - x_clr) * 
                    highCld_anom.groupby('time.month'))
    
    # Get anomalous cloud fraction
    cld_frac_anom   = get_anomaly(cld_area.sel(press=slice(2,6)),1)
    highCld_frac_anom = cld_frac_anom.sum(dim=('press','opt'))
    highCld_ratio     = highCld_frac_anom / highCld_frac_clim
    
    # Get cloud fraction anomaly that includes changes in p-tau distribution
    adjusted_cld_frac_anom = cld_frac_anom.groupby('time.month') - cld_frac_clim.sel(press=slice(2,6)) * highCld_ratio
    
    ### Get the radiative perturbation due to changes in the p-tau distribution
    dR_dcld_dist = ((x_cld - x_clr) * adjusted_cld_frac_anom.groupby('time.month')).sum(dim=('press','opt'))
        
    return xr.Dataset(data_vars = {'dR_dcld_frac':dR_dcld_frac, 'dR_dcld_dist':dR_dcld_dist})

In [ ]:
## Get low cloud and high cloud cre from FBCT
def calc_fbct_cre(lw_flux_cld,lw_flux_clr,sw_flux_cld,sw_flux_clr,cld_frac,cld_type):
    # Get net DOWNWARD flux and then the climatological mean
    net_flux_all = (-sw_flux_cld - lw_flux_cld).groupby('time.month').mean(dim='time')
    net_flux_clr = (-sw_flux_clr - lw_flux_clr).groupby('time.month').mean(dim='time')

    # Downsample the net DOWNWARD flux
    net_flux_all_loRes = net_flux_all.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
    net_flux_clr_loRes = net_flux_clr.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
    
    # Downsample the cloud fraction
    cld_frac_loRes = cld_frac.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
#     cld_frac_anom = get_anomaly(cld_frac_loRes,1)
    
    # Finally, sum across p,tau bins to get monthly cloud radiative anomaly *** (R_pt - R_clr)*f'_pt ***
    if cld_type == 'low':
        get_cre = get_mean_across_bins_loCld(net_flux_all_loRes,net_flux_clr_loRes,cld_frac_loRes)
    elif cld_type == 'high':
        get_cre = get_mean_across_bins_highCld(net_flux_all_loRes,net_flux_clr_loRes,cld_frac_loRes)
    
    return get_cre

In [ ]:
## Get net cre, clear-sky, and all-sky from EBAF
def calc_net_cre():
    net_flux_cld = EBAF_flux.toa_net_all_mon
    net_flux_clr = EBAF_flux.toa_net_clr_c_mon
    
    net_flux_cld_loRes = net_flux_cld.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
    net_flux_clr_loRes = net_flux_clr.interp(latitude=new_lat_medRes, longitude=new_lon_medRes)
    
    net_cre_loRes = net_flux_cld_loRes - net_flux_clr_loRes
    calc_net = get_anomaly(net_cre_loRes,1)
    
    return calc_net

def calc_ebaf_rad_anom(data):
    ebaf_rad_anom = get_anomaly(data,1)
    ebaf_rad_anom_loRes = ebaf_rad_anom.interp(latitude=new_lat_medRes,longitude=new_lon_medRes)
    
    return ebaf_rad_anom_loRes